### Code to apply the hypothesis to the predictions of the model
- Update the file initials to proceed
- If you want to apply the hypothesis to a file containing single task predictions then update the Index constants accordingly

In [ ]:
# Import Libraries

import json
from collections import Counter

In [ ]:
# File Initials

initial_predictions = "prediction.txt"
file_initials_task2 = "MaSaC_test_efr"
file_initials_task3 = "MELD_test_efr"

In [ ]:
# Constants denoting line number in final file for each task

IX1_BEGIN = 0
IX1_END = 1579
IX2_BEGIN = 1580
IX2_END = 9269
IX3_BEGIN = 9270
IX3_END = 17911

In [ ]:
# Read Json Files

file_name_task2 = "./" + file_initials_task2 + ".json"
with open(file_name_task2, 'r') as file:
    dataset_task2 = json.load(file)

file_name_task3 = "./" + file_initials_task3 + ".json"
with open(file_name_task3, 'r') as file:
    dataset_task3 = json.load(file)

In [ ]:
# Read the data of the prediction file

f = open(initial_predictions, 'r')
soln = []
# Task 1 - MaSaC ERC
for i in range(IX1_BEGIN, IX1_END+1):
    x = f.readline()
    soln.append(x[:-1])

# Task 2 - MaSaC EFR
for i in range(IX2_BEGIN, IX2_END+1):
    x = f.readline()
    soln.append(x[:-1])

# Task 3 - MELD EFR
for i in range(IX3_BEGIN, IX3_END+1):
    x = f.readline()
    soln.append(x[:-1])

f.close()

In [ ]:
def get_lengths(dataset):

    # Reading the data

    init_len = []

    speakers_list = []
    utterances_list = []

    for i in range(len(dataset)):
        speakers_list.append(dataset[i]['speakers'])
        utterances_list.append(dataset[i]['utterances'])
        init_len.append(len(dataset[i]['emotions']))

    N = len(dataset)

    # Calculate the new length of each converation according to the hypothesis

    new_len = []

    for i in range(len(dataset)):
        count_utt = init_len[i]
        target_speaker = speakers_list[i][count_utt-1]

    for j in range(count_utt):
        ix = count_utt - 1 - j

        if ix != count_utt - 1:

            if speakers_list[i][ix] == target_speaker:
                new_len.append(min(5, j+1))
                break

    return init_len, new_len

In [ ]:
# Visualising the distribution of lengths

def print_distr(len):

    counter = Counter(len)
    print(counter)

In [ ]:
# Applying the hypothesis

def apply_hypothesis(IX_BEGIN, IX_END, dataset, soln):

    # Get the lengths
    init_len, new_len = get_lengths(dataset)

    # Print the distributions
    print("Initial Length")
    print_distr(init_len)
    print("New Length")
    print_distr(new_len)

    # Mask the predictions to 0.0
    i = 0
    changes = 0
    ix = IX_BEGIN

    while ix < IX_END+1:

        count_utt = init_len[i]
        count_pred = new_len[i]
        init_zeros = count_utt - count_pred

        for j in range(init_zeros):
            if soln[ix] == '1.0':
                changes += 1
            soln[ix] = '0.0'
            ix += 1

        ix += count_utt-init_zeros
        i += 1

    print(changes, " predictions were converted from 1.0 to 0.0")

In [ ]:
# Applying Hypothesis

apply_hypothesis(IX2_BEGIN, IX2_END, dataset_task2, soln)
apply_hypothesis(IX2_BEGIN, IX2_END, dataset_task3, soln)

In [ ]:
# Write the updated predictions into a new file

f = open("new_predictions.txt", 'w+')

# Task 1 - MaSaC ERC
for i in range(IX1_BEGIN, IX1_END+1):
    f.write(str(soln[i])+'\n')
    print(soln[i])

# Task 2 - MaSaC EFR
for i in range(IX2_BEGIN, IX2_END+1):
    f.write(str(soln[i])+'\n')
    print(soln[i])

# Task 3 - MELD EFR
for i in range(IX3_BEGIN, IX3_END+1):
    f.write(str(soln[i])+'\n')

f.close()